In [ ]:
!pip install langchain faiss-cpu openai streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall langchain -y
!pip install langchain==0.0.171
!pip install sentence-transformers

Found existing installation: langchain 0.3.7
Uninstalling langchain-0.3.7:
  Successfully uninstalled langchain-0.3.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [ ]:
import json
import os
import faiss
import numpy as np
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI


**Reading the JSON file**

In [ ]:

# Define the path to the folder containing JSON files
json_folder_path = '/content/datasets'

# Function to load all JSON files in a folder
def load_json_files_from_folder(folder_path):
    all_product_data = []

    # Loop through each file in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):  # Make sure it’s a JSON file
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                product_data = json.load(f)
                all_product_data.append(product_data)

    return all_product_data

# Load all JSON files
product_data_list = load_json_files_from_folder(json_folder_path)

# Print the first product data to verify the content
print(json.dumps(product_data_list[0], indent=2))


{
  "HIGHLIGHTS OF PRESCRIBING INFORMATION": "These highlights do not include all the information needed to use TRAVOPROST OPHTHALMIC SOLUTION (Ionic Buffered Solution) safely and effectively. See full prescribing information for TRAVOPROST OPHTHALMIC SOLUTION (Ionic Buffered Solution).TRAVOPROST ophthalmic solution (ionic buffered solution), for topical ophthalmic useInitial U.S. Approval: 2001INDICATIONS AND USAGETravoprost ophthalmic solution 0.004% (ionic buffered solution) is a prostaglandin analog indicated for the reduction of elevated intraocular pressure in patients with open-angle glaucoma or ocular hypertension. (1)DOSAGE AND ADMINISTRATIONOne drop in the affected eye(s) once daily in the evening. (2)DOSAGE FORMS AND STRENGTHSOphthalmic solution containing travoprost 0.04 mg/mL. (3)CONTRAINDICATIONSNone (4)WARNINGS AND PRECAUTIONSPigmentation: Pigmentation of the iris, periorbital tissue (eyelid), and eyelashes can occur. Iris pigmentation likely to be permanent. (5.1)Eyelas

In [ ]:
def extract_useful_information(product_data):
    useful_info = {
        "product_name": product_data.get("product_name", ""),
        "highlights": product_data.get("HIGHLIGHTS OF PRESCRIBING INFORMATION", ""),
        "indications_and_usage": product_data.get("1 INDICATIONS AND USAGE", ""),
        "dosage_and_administration": product_data.get("2 DOSAGE AND ADMINISTRATION", ""),
        "contraindications": product_data.get("4 CONTRAINDICATIONS", ""),
        "warnings": product_data.get("5 WARNINGS AND PRECAUTIONS", ""),
        "adverse_reactions": product_data.get("6 ADVERSE REACTIONS", ""),
        "drug_interactions": product_data.get("7 DRUG INTERACTIONS", ""),
        "clinical_pharmacology": product_data.get("12 CLINICAL PHARMACOLOGY", ""),
        "how_supplied": product_data.get("16 HOW SUPPLIED/STORAGE AND HANDLING", "")
    }
    return useful_info

# Extract useful information for each product
product_info_list = [extract_useful_information(product_data) for product_data in product_data_list]

# Print the first product's extracted information
print(json.dumps(product_info_list[35], indent=2))


{
  "product_name": "Piroxicam Capsules, USP",
  "highlights": "These highlights do not include all the information needed to use PIROXICAM CAPSULES safely and effectively. See full prescribing information for PIROXICAM CAPSULES.PIROXICAM capsules, for oral useInitial U.S. Approval: 1982WARNING: RISK OF SERIOUS CARDIOVASCULAR AND GASTROINTESTINAL EVENTSSee full prescribing information for complete boxed warning.Nonsteroidal anti-inflammatory drugs (NSAIDs) cause an increased risk of serious cardiovascular thrombotic events, including myocardial infarction and stroke, which can be fatal. This risk may occur early in treatment and may increase with duration of use (5.1)Piroxicam is contraindicated in the setting of coronary artery bypass graft (CABG) surgery (4,5.1)NSAIDs cause an increased risk of serious gastrointestinal (GI) adverse events including bleeding, ulceration, and perforation of the stomach or intestines, which can be fatal. These events can occur at any time during use and

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize SentenceTransformer with a pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to convert product data to embeddings
def create_embeddings_from_texts(texts):
    return model.encode(texts, convert_to_tensor=True)

# Extract text content from all product information
all_texts = [item["indications_and_usage"] + "\n" + item["dosage_and_administration"] + "\n" + item["warnings"]
             for item in product_info_list]

# Generate embeddings for all product texts
embeddings = create_embeddings_from_texts(all_texts)

# Move embeddings to CPU before converting to NumPy
embeddings = embeddings.cpu() # Move embeddings to CPU

# Convert embeddings to numpy arrays for FAISS
embeddings_np = np.array(embeddings)

# Normalize the embeddings (required by FAISS)
faiss.normalize_L2(embeddings_np)

# Create FAISS index
faiss_index = faiss.IndexFlatL2(embeddings_np.shape[1])  # L2 distance for similarity
faiss_index.add(embeddings_np)

# Check the FAISS index
print(faiss_index)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7814b8334930> >


In [ ]:
# Function to query FAISS index
def query_faiss_index(query, faiss_index, model):
    # Encode the query using the SentenceTransformer model
    query_embedding = model.encode([query], convert_to_tensor=True)

    # Move the query tensor to CPU and convert to numpy
    query_embedding_np = query_embedding.cpu().numpy()

    # Normalize the query embedding (required by FAISS)
    faiss.normalize_L2(query_embedding_np)

    # Perform the search
    _, indices = faiss_index.search(query_embedding_np, k=1)  # k=1 for top match

    # Retrieve the most relevant text based on the closest match
    return all_texts[indices[0][0]]

# Example query
query = "What is Acetazolamide used for?"
response = query_faiss_index(query, faiss_index, model)
print(response)


Dorzolamide hydrochloride ophthalmic solution is indicated in the treatment of elevated intraocular pressure in patients with ocular hypertension or open-angle glaucoma.
The dose is one drop of dorzolamide hydrochloride ophthalmic solution in the affected eye(s) three times daily. Dorzolamide hydrochloride ophthalmic solution may be used concomitantly with other topical ophthalmic drug products to lower intraocular pressure. If more than one topical ophthalmic drug is being used, the drugs should be administered at least five minutes apart.
5.1 Sulfonamide HypersensitivityDorzolamide hydrochloride ophthalmic solution contains dorzolamide, a sulfonamide; and although administered topically, it is absorbed systemically. Therefore, the same types of adverse reactions that are attributable to sulfonamides may occur with topical administration of dorzolamide hydrochloride ophthalmic solution. Fatalities have occurred, although rarely, due to severe reactions to sulfonamides including Steven

**Summarization of dataset**

In [ ]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print("Summarization Pipeline loaded.")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Summarization Pipeline loaded.


In [ ]:
def summarize_text(text: str, max_length: int = 130, min_length: int = 30) -> str:
    try:
        # Generate summary
        summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
        return summary[0]["summary_text"]
    except Exception as e:
        return f"Error: {str(e)}"



# Example query
query = "What is Acetazolamide used for?"
response = query_faiss_index(query, faiss_index, model)
long_text = response
summary = summarize_text(response)
print(f"Summary:\n{summary}")

Summary:
Dorzolamide hydrochloride ophthalmic solution is indicated in the treatment of elevated intraocular pressure in patients with ocular hypertension or open-angle glaucoma. The dose is one drop in the affected eye(s) three times daily. Fatalities have occurred, although rarely, due to severe reactions to sulfonamides.


In [ ]:
from transformers import pipeline

# Load the GPT-2 model for text generation
qa_pipeline = pipeline("text-generation", model="gpt2", tokenizer="gpt2")


def answer_question(question: str) -> str:
    try:
        # Format the question in a way that T5 understands
        input_text = f"question: {question} answer:"
        response = qa_pipeline(input_text, max_length=100, num_return_sequences=1)
        return response[0]["generated_text"].strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
question_text = "What is the use of Paracetamol?"
answer = answer_question(question_text)
print(f"Q: {question_text}\nA: {answer}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Q: What is the use of Paracetamol?
A: question: What is the use of Paracetamol? answer: Paracetamol is an analgesic, anti-depressant, anti-inflammatory, and anti-inflammatory compound that's also used to treat diabetes. Why is Paracetamol the key to getting a better cholesterol picture and improving liver function when there is no risk of diabetes? Answer: Paracetamol is extremely helpful in getting cholesterol levels back on the right track and it's also a low-risk, quick


In [ ]:
med_list = [
    "Acetazolamide Extended-Release Capsules",
    "Amlodipine Besylate and Olmesartan Medoxomil Tablets",
    "Amoxicillin Capsules, USP",
    "Amoxicillin and Clavulanate Potassium for Oral Suspension, USP",
    "Amoxicillin and Clavulanate Potassium Tablets, USP",
    "Aspirin and Extended-Release Dipyridamole Capsules",
    "Atorvastatin Calcium Tablets",
    "Bimatoprost Ophthalmic Solution",
    "Celecoxib Capsules",
    "Chlordiazepoxide Hydrochloride and Clidinium Bromide Capsules, USP",
    "Clindamycin Hydrochloride Capsules, USP",
    "Clobazam Tablets",
    "Clobetasol Propionate Topical Solution, USP",
    "Clomipramine Hydrochloride Capsules, USP",
    "Cromolyn Sodium Inhalation Solution, USP",
    "Cromolyn Sodium Oral Solution",
    "Dalfampridine Extended-Release Tablets",
    "Diclofenac Sodium and Misoprostol Delayed-Release Tablets, USP",
    "Dorzolamide HCl Ophthalmic Solution, USP",
    "Dorzolamide HCl and Timolol Maleate Ophthalmic Solution, USP",
    "Dorzolamide HCl and Timolol Maleate Ophthalmic Solution, USP (Preservative-Free)",
    "Erythromycin Topical Solution, USP",
    "Famotidine for Oral Suspension, USP",
    "Fenofibric Acid Delayed-Release Capsules",
    "Glimepiride Tablets, USP",
    "Ketorolac Tromethamine Ophthalmic Solution",
    "Levocetirizine Dihydrochloride Tablets, USP",
    "Mefenamic Acid Capsules, USP",
    "Metformin Hydrochloride Extended-Release Tablets, USP",
    "Metformin Hydrochloride Oral Solution",
    "Methenamine Hippurate Tablets, USP",
    "Olmesartan Medoxomil Tablets, USP",
    "Piroxicam Capsules, USP",
    "Potassium Chloride Oral Solution, USP",
    "Ramelteon Tablets",
    "Ranolazine Extended-Release Tablets",
    "Rasagiline Tablets",
    "Roflumilast Tablets",
    "Rufinamide Tablets, USP",
    "Tafluprost Ophthalmic Solution",
    "Telmisartan Tablets, USP",
    "Timolol Maleate Ophthalmic Solution, USP (Preservative-Free)",
    "Tobramycin Inhalation Solution, USP",
    "Travoprost Ophthalmic Solution, USP",
    "Triamcinolone Acetonide Lotion, USP"
]
def handle_query(query, product_data_list):
    """
    This function decides whether to use RAG summarization or QA answering module based on the query type.
    """

    # Check if the query matches any of the specific types
    if any(keyword.lower() in query.lower() for keyword in med_list):
        print("Running RAG Summarization for Type 5 Query...")
        return query_faiss_index(query, faiss_index, model)

    else:
        # For other queries, use the Question-Answering Module
        print("Running QA Module for general query...")
        answer = answer_question(question_text)
        return answer
#print(f"Q: {question_text}\nA: {answer}")

answer = handle_query(query, product_data_list)
print(answer)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Running QA Module for general query...
question: What is the primary use of Paracetamol? answer: No other drug in the entire series has been studied on the brain. How much Paracetamol does you need depends on the drug, the type of the drug and the level of side-effects. (Yes, a drug of note like Paracetamol is better to use as compared with a tranquilizer like Ritalin and an antidepressant which has been shown to be effective in a number of different conditions and in


In [ ]:
# # # Install SentenceTransformers library
# # !pip install sentence-transformers

# from sentence_transformers import SentenceTransformer
# import faiss
# import numpy as np
# import json
# import os

# # Initialize SentenceTransformer with a pre-trained model
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # Define the path to the folder containing JSON files
# json_folder_path = '/content/datasets'

# # Function to load all JSON files in a folder
# def load_json_files_from_folder(folder_path):
#     all_product_data = []
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.json'):  # Make sure it’s a JSON file
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 product_data = json.load(f)
#                 all_product_data.append(product_data)
#     return all_product_data

# # Function to extract useful information from product data
# def extract_useful_information(product_data):
#     useful_info = {
#         "product_name": product_data.get("product_name", ""),
#         "highlights": product_data.get("HIGHLIGHTS OF PRESCRIBING INFORMATION", ""),
#         "indications_and_usage": product_data.get("1 INDICATIONS AND USAGE", ""),
#         "dosage_and_administration": product_data.get("2 DOSAGE AND ADMINISTRATION", ""),
#         "contraindications": product_data.get("4 CONTRAINDICATIONS", ""),
#         "warnings": product_data.get("5 WARNINGS AND PRECAUTIONS", ""),
#         "adverse_reactions": product_data.get("6 ADVERSE REACTIONS", ""),
#         "drug_interactions": product_data.get("7 DRUG INTERACTIONS", ""),
#         "clinical_pharmacology": product_data.get("12 CLINICAL PHARMACOLOGY", ""),
#         "how_supplied": product_data.get("16 HOW SUPPLIED/STORAGE AND HANDLING", "")
#     }
#     return useful_info

# # Load all JSON files
# product_data_list = load_json_files_from_folder(json_folder_path)

# # Extract useful information for each product
# product_info_list = [extract_useful_information(product_data) for product_data in product_data_list]

# # Extract text content from all product information
# all_texts = [item["indications_and_usage"] + "\n" + item["dosage_and_administration"] + "\n" + item["warnings"]
#              for item in product_info_list]

# # Generate embeddings for all product texts
# embeddings = model.encode(all_texts, convert_to_tensor=True)

# # Move embeddings to CPU if they are on the GPU
# embeddings_cpu = embeddings.cpu().numpy()  # Ensure that the embeddings are on the CPU and then convert to numpy

# # Create FAISS index
# faiss_index = faiss.IndexFlatL2(embeddings_cpu.shape[1])  # L2 distance for similarity
# faiss_index.add(embeddings_cpu)

# # Function to query FAISS index
# def query_faiss_index(query, faiss_index, model):
#     # Encode the query using the SentenceTransformer model
#     query_embedding = model.encode([query], convert_to_tensor=True)

#     # Move the query tensor to CPU and convert to numpy
#     query_embedding_np = query_embedding.cpu().numpy()

#     # Normalize the query embedding (required by FAISS)
#     faiss.normalize_L2(query_embedding_np)

#     # Perform the search
#     _, indices = faiss_index.search(query_embedding_np, k=1)  # k=1 for top match

#     # Retrieve the most relevant text based on the closest match
#     return all_texts[indices[0][0]]

# # Example query
# query = "What is the warning of aspirin?"
# response = query_faiss_index(query, faiss_index, model)
# print(response)


In [ ]:
from pyngrok import ngrok

# Add your ngrok authentication token
!ngrok config add-authtoken 2pkKqZ1cTx2eMLKFSsoecyKj4jr_5hFEtFR9VC6ueZCY1jh9m

# Start ngrok tunnel for HTTP
public_url = ngrok.connect(8501, "http")  # Correct usage without `port=` keyword
print(f"Streamlit app is live at: {public_url}")

# Run Streamlit in the foreground to see the output
!streamlit run app.py


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://3101-34-87-113-35.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.113.35:8501

2024-12-04 12:20:25.716821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 12:20:25.740099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 12:20:25.747508: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 1